# Fully Connected Autoencoder Example
modified from https://blog.keras.io/building-autoencoders-in-keras.html

## Prepare Data Set

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
(x_train, _),(x_test, _) = mnist.load_data()

## Check Data 

In [ ]:
import matplotlib.pyplot as plt
print('x_train shape: {}  x_test shape: {}'.format(np.shape(x_train),np.shape(x_test)))
for i in range(16):
    plt.subplot(4, 4, 1 + i, xticks=[], yticks=[])
    img_id = np.random.randint(np.shape(x_train)[0])
    im = x_train[img_id,::]
    plt.imshow(im)
    plt.gray()

## Preprocessing Data

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

## Build Model 

In [ ]:
from tensorflow.keras.layers import Dense
autoencoder = tf.keras.Sequential()
#Encoder
autoencoder.add(Dense(128,activation='relu',input_shape=(784,)))
autoencoder.add(Dense(64,activation='relu'))

#Compressed representation
autoencoder.add(Dense(32,activation='relu'))

#Decoder
autoencoder.add(Dense(64,activation='relu'))
autoencoder.add(Dense(128,activation='relu'))
autoencoder.add(Dense(784,activation='sigmoid'))

autoencoder.summary()

## Train Model

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

## Evaluate Model

In [ ]:
x_recon = autoencoder.predict(x_test)
for i in range(16):
    plt.subplot(4, 8, 1 + i*2, xticks=[], yticks=[])
    img_id = np.random.randint(np.shape(x_test)[0])
    im = x_test[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()
    plt.title('Ori')
    plt.subplot(4, 8, 2 + i*2, xticks=[], yticks=[])
    
    im = x_recon[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()
    plt.title('AE')

# Denosing with Autoencoder 

## Add noise to data

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

## Check Data 

In [ ]:
for i in range(16):
    plt.subplot(4, 4, 1 + i, xticks=[], yticks=[])
    img_id = np.random.randint(np.shape(x_train)[0])
    im = x_train_noisy[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()

## Create a New Model for Denosing

In [ ]:
tf.reset_default_graph()
tf.keras.backend.clear_session()
autoencoder = tf.keras.Sequential()
#Encoder
autoencoder.add(Dense(128,activation='relu',input_shape=(784,)))
autoencoder.add(Dense(64,activation='relu'))

#Compressed representation
autoencoder.add(Dense(32,activation='relu'))

#Decoder
autoencoder.add(Dense(64,activation='relu'))
autoencoder.add(Dense(128,activation='relu'))
autoencoder.add(Dense(784,activation='sigmoid'))

#Train
autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

## Evaluate Model

In [ ]:
x_denoise = autoencoder.predict(x_test_noisy)
for i in range(16):
    plt.subplot(4, 8, 1 + i*2, xticks=[], yticks=[])
    img_id = np.random.randint(np.shape(x_test_noisy)[0])
    im = x_test_noisy[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()
    plt.title('Ori')
    plt.subplot(4, 8, 2 + i*2, xticks=[], yticks=[])
    
    im = x_denoise[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()
    plt.title('AE')